In [1]:
import pandas as pd
import numpy as np
import os
from datetime import date, datetime, timedelta
from pathlib import Path

import copy
import pyarrow.parquet as pq

In [2]:
from technicals import *
from tsfresh import extract_features

In [3]:
data_path = Path('../data')
data_path.exists()

True

<h3> concatenate book data for train and test</h3>

In [4]:
dest_folder ='z_1'

In [5]:
train_book_path = data_path.joinpath('book_train.parquet')
book_train_sub = next(os.walk(train_book_path))[1]

In [6]:
for sub_f_ in book_train_sub:
    file_loc= train_book_path.joinpath(sub_f_)
    df = pq.read_table(file_loc).to_pandas()
    
    #cal wap and spread
    df['wap']=cal_wap(df)
    df['cal_spread1']=cal_spread1(df)
    df['cal_spread2']=cal_spread2(df)

    #cal technical indicators by time_id
    sum_list = []
    for time_id_ in df['time_id'].unique().tolist():
        series_ = df.loc[df['time_id']==time_id_, 'wap']
        item_ = [time_id_, cal_ema(series_), cal_rsi(series_), cal_ssma(series_), cal_dema(series_), 
                 cal_tema(series_), cal_trix(series_), cal_er(series_), cal_smma(series_), 
                 cal_roc(series_), cal_stochrsi(series_)]
        sum_list.append(item_)

    header =['time_id', 'wap_ema', 'wap_rsi', 'wap_ssma', 'wap_dema', 
                 'wap_tema', 'wap_trix', 'wap_er', 'wap_smma', 
                 'wap_roc', 'wap_stochrsi']
    df_pivot = pd.DataFrame(sum_list, columns=header)

    #cal realized volatility
    df['log_return']=df.groupby(['time_id'])['wap'].apply(cal_log_return)
    df_ = df.loc[df['log_return'].notna(), ['time_id', 'log_return']].copy(deep=True)
    df_real_vol_ =  df_.groupby(['time_id'])['log_return'].agg(cal_real_vol).reset_index()
    df_real_vol_.rename(columns = {'log_return':'real_vol'}, inplace=True)

    print(df_pivot.shape)
    df_pivot = df_pivot.merge(df_real_vol_, how='inner', on=['time_id'])
    print(df_pivot.shape)
    del df_real_vol_, df_

    #calculate basic summary stats
    #min, max, med
    df_ = df.pivot_table(index=['time_id'], values=['wap', 'cal_spread1'], 
                   aggfunc={np.mean, np.median, np.max, np.min, np.std
                           }
                  )

    new_cols = []
    for c_ in df_.columns:
        new_cols.append(f'{c_[0]}_{c_[1]}')

    df_.columns=new_cols
    df_.reset_index(inplace=True)


    print(df_pivot.shape)
    df_pivot = df_pivot.merge(df_, how='inner', on=['time_id'])
    print(df_pivot.shape)
    del df_

    #get tsfresh features
    sel_cols=['time_id', 'seconds_in_bucket', 'wap']
    df_features = extract_features(df[sel_cols], column_id="time_id", column_sort="seconds_in_bucket")
    df_features['time_id']=df_features.index

    #merge all into one dataframe
    print(df_features.shape, df_pivot.shape)
    df_pivot = df_pivot.merge(df_features, on='time_id', how='inner')
    print(df_pivot.shape)
    del df_features


    #assign stock id and file
    stock_id=sub_f_.split('=')[-1]  
    df_pivot['stock_id']=stock_id        
    df_pivot['file']=str(file_loc)

    #save data
    df_pivot.to_csv(f'../data/{dest_folder}/{stock_id}', sep='|', compression='bz2')
    
    del df_pivot, df

(3830, 11)
(3830, 12)
(3830, 12)
(3830, 22)


Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 20/20 [01:33<00:00,  4.68s/it]


(3830, 788) (3830, 22)
(3830, 809)
(3830, 11)
(3830, 12)
(3830, 12)
(3830, 22)


Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 20/20 [02:40<00:00,  8.00s/it]


(3830, 788) (3830, 22)
(3830, 809)
(3830, 11)
(3830, 12)
(3830, 12)
(3830, 22)


...

(3830, 788) (3830, 22)
(3830, 809)
(3830, 11)
(3830, 12)
(3830, 12)
(3830, 22)


Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 20/20 [01:33<00:00,  4.66s/it]


(3830, 788) (3830, 22)
(3830, 809)
(3830, 11)
(3830, 12)
(3830, 12)
(3830, 22)


Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 20/20 [03:21<00:00, 10.06s/it]


(3830, 788) (3830, 22)
(3830, 809)
